In [2]:
!pip install yfinance
!pip install transformers
!pip install beautifulsoup4


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [2]:
import yfinance as yf
import datetime
import numpy as np
import pandas as pd
from pylab import mpl, plt
import math, time
import itertools
from datetime import datetime
from operator import itemgetter
from tqdm import tqdm
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable
from transformers import RobertaTokenizer, RobertaModel
import torch.nn.functional as F

/Users/visriv/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [1]:
import wandb
wandb.login()
wandb.init(project="stock_prediction")

wandb: Currently logged in as: visriv. Use `wandb login --relogin` to force relogin


### Hyperparams

In [57]:
no_of_days_to_lookforward = 1
no_of_days_to_lookback = 5
up_threshold = 0.015
down_threshold = -0.015
max_text_per_iter = 20
batch_size = 16
MAX_LEN = 10
num_epochs = 200

### Get stocks data for last N days

In [5]:
stock_symbols = [ 'XOM']
no_of_days = 4*365

EXPORT_DATA_FOLDER = './data/'

# Set the start and end dates for the data 
# here matching it with dates of news text available
train_start = datetime.strptime('2020/01/04', '%Y/%m/%d')
train_end = datetime.strptime('2022/09/30', '%Y/%m/%d')
test_start = datetime.strptime('2022/10/01', '%Y/%m/%d')
test_end = datetime.strptime('2023/01/04', '%Y/%m/%d')


# start = datetime.datetime.now() - datetime.timedelta(days=no_of_days)
# end = datetime.datetime.now()

# Get training data
for symbol in stock_symbols:
    # Download the historical price and volume data using yfinance
    train_data_raw = yf.download(symbol, start=train_start, end=train_end)

    # Normalize features by percent of changes between today and yesterday
    pct_change_open = train_data_raw['Open'].pct_change().fillna(0)
    pct_change_high = train_data_raw['High'].pct_change().fillna(0)
    pct_change_high_over_open = (train_data_raw['High']-train_data_raw['Open'])/train_data_raw['Open']
    pct_change_low = train_data_raw['Low'].pct_change().fillna(0)
    pct_change_low_over_open = (train_data_raw['Low']-train_data_raw['Open'])/train_data_raw['Open']
    pct_change_close = train_data_raw['Close'].pct_change().fillna(0)
    pct_change_close_over_open = (train_data_raw['Close']-train_data_raw['Open'])/train_data_raw['Open']
    pct_change_adjclose = train_data_raw['Adj Close'].pct_change().fillna(0)
    pct_change_adjclose_over_open = (train_data_raw['Adj Close']-train_data_raw['Open'])/train_data_raw['Open']
    pct_change_volume = train_data_raw['Volume'].pct_change().fillna(0)

    # Prepare labels: 2 means the close price of tomorow is higher than today's close price; 1 is down; 0 means the movement is between up_threshold and down_threshold
    label = np.where(pct_change_close > up_threshold, 2, np.where(pct_change_close < down_threshold, 1, 0))[1:]
    label = np.append(label, 0)

    # Construct a train_data_norm data frame
    train_data_norm = pd.DataFrame({'Open_norm':pct_change_open,
                              'High_norm':pct_change_high,
                              'Low_norm': pct_change_low,
                              'Close_norm':pct_change_close,
                              'Volume_norm':pct_change_volume,
                              'High-Open_norm':pct_change_high_over_open,
                              'Low-Open_norm':pct_change_low_over_open,
                              'Close-Open_norm':pct_change_close_over_open,
                              'Label_2up1down':label})

    # Normalize by min-max normalization after the pct normalization
    train_data_norm['Open_norm'] = train_data_norm['Open_norm'].apply(lambda x: (x - train_data_norm['Open_norm'].min()) / (train_data_norm['Open_norm'].max() - train_data_norm['Open_norm'].min()))
    train_data_norm['High_norm'] = train_data_norm['High_norm'].apply(lambda x: (x - train_data_norm['High_norm'].min()) / (train_data_norm['High_norm'].max() - train_data_norm['High_norm'].min()))
    train_data_norm['Low_norm'] = train_data_norm['Low_norm'].apply(lambda x: (x - train_data_norm['Low_norm'].min()) / (train_data_norm['Low_norm'].max() - train_data_norm['Low_norm'].min()))
    train_data_norm['Close_norm'] = train_data_norm['Close_norm'].apply(lambda x: (x - train_data_norm['Close_norm'].min()) / (train_data_norm['Close_norm'].max() - train_data_norm['Close_norm'].min()))
    train_data_norm['Volume_norm'] = train_data_norm['Volume_norm'].apply(lambda x: (x - train_data_norm['Volume_norm'].min()) / (train_data_norm['Volume_norm'].max() - train_data_norm['Volume_norm'].min()))
    train_data_norm['High-Open_norm'] = train_data_norm['High-Open_norm'].apply(lambda x: (x - train_data_norm['High-Open_norm'].min()) / (train_data_norm['High-Open_norm'].max() - train_data_norm['High-Open_norm'].min()))
    train_data_norm['Low-Open_norm'] = train_data_norm['Low-Open_norm'].apply(lambda x: (x - train_data_norm['Low-Open_norm'].min()) / (train_data_norm['Low-Open_norm'].max() - train_data_norm['Low-Open_norm'].min()))
    train_data_norm['Close-Open_norm'] = train_data_norm['Close-Open_norm'].apply(lambda x: (x - train_data_norm['Close-Open_norm'].min()) / (train_data_norm['Close-Open_norm'].max() - train_data_norm['Close-Open_norm'].min()))

    # Remove the first and the last row, becuase of NAN values
    train_data_raw = train_data_raw.iloc[1:-1]
    train_data_norm = train_data_norm.iloc[1:-1]

    train_data_raw.to_csv(EXPORT_DATA_FOLDER+symbol+'train_raw_data.csv', index=True)
    train_data_norm.to_csv(EXPORT_DATA_FOLDER+symbol+'train_norm_data.csv', index=True)
    
    
# Get test data
for symbol in stock_symbols:
    # Download the historical price and volume data using yfinance
    test_data_raw = yf.download(symbol, start=test_start, end=test_end)

    # Normalize features by percent of changes between today and yesterday
    pct_change_open = test_data_raw['Open'].pct_change().fillna(0)
    pct_change_high = test_data_raw['High'].pct_change().fillna(0)
    pct_change_high_over_open = (test_data_raw['High']-test_data_raw['Open'])/test_data_raw['Open']
    pct_change_low = test_data_raw['Low'].pct_change().fillna(0)
    pct_change_low_over_open = (test_data_raw['Low']-test_data_raw['Open'])/test_data_raw['Open']
    pct_change_close = test_data_raw['Close'].pct_change().fillna(0)
    pct_change_close_over_open = (test_data_raw['Close']-test_data_raw['Open'])/test_data_raw['Open']
    pct_change_adjclose = test_data_raw['Adj Close'].pct_change().fillna(0)
    pct_change_adjclose_over_open = (test_data_raw['Adj Close']-test_data_raw['Open'])/test_data_raw['Open']
    pct_change_volume = test_data_raw['Volume'].pct_change().fillna(0)

    # Prepare labels: 2 means the close price of tomorow is higher than today's close price; 1 is down; 0 means the movement is between up_threshold and down_threshold
    label = np.where(pct_change_close > up_threshold, 2, np.where(pct_change_close < down_threshold, 1, 0))[1:]
    label = np.append(label, 0)

    # Construct a test_data_norm data frame
    test_data_norm = pd.DataFrame({'Open_norm':pct_change_open,
                              'High_norm':pct_change_high,
                              'Low_norm': pct_change_low,
                              'Close_norm':pct_change_close,
                              'Volume_norm':pct_change_volume,
                              'High-Open_norm':pct_change_high_over_open,
                              'Low-Open_norm':pct_change_low_over_open,
                              'Close-Open_norm':pct_change_close_over_open,
                              'Label_2up1down':label})

    # Normalize by min-max normalization after the pct normalization
    test_data_norm['Open_norm'] = test_data_norm['Open_norm'].apply(lambda x: (x - test_data_norm['Open_norm'].min()) / (test_data_norm['Open_norm'].max() - test_data_norm['Open_norm'].min()))
    test_data_norm['High_norm'] = test_data_norm['High_norm'].apply(lambda x: (x - test_data_norm['High_norm'].min()) / (test_data_norm['High_norm'].max() - test_data_norm['High_norm'].min()))
    test_data_norm['Low_norm'] = test_data_norm['Low_norm'].apply(lambda x: (x - test_data_norm['Low_norm'].min()) / (test_data_norm['Low_norm'].max() - test_data_norm['Low_norm'].min()))
    test_data_norm['Close_norm'] = test_data_norm['Close_norm'].apply(lambda x: (x - test_data_norm['Close_norm'].min()) / (test_data_norm['Close_norm'].max() - test_data_norm['Close_norm'].min()))
    test_data_norm['Volume_norm'] = test_data_norm['Volume_norm'].apply(lambda x: (x - test_data_norm['Volume_norm'].min()) / (test_data_norm['Volume_norm'].max() - test_data_norm['Volume_norm'].min()))
    test_data_norm['High-Open_norm'] = test_data_norm['High-Open_norm'].apply(lambda x: (x - test_data_norm['High-Open_norm'].min()) / (test_data_norm['High-Open_norm'].max() - test_data_norm['High-Open_norm'].min()))
    test_data_norm['Low-Open_norm'] = test_data_norm['Low-Open_norm'].apply(lambda x: (x - test_data_norm['Low-Open_norm'].min()) / (test_data_norm['Low-Open_norm'].max() - test_data_norm['Low-Open_norm'].min()))
    test_data_norm['Close-Open_norm'] = test_data_norm['Close-Open_norm'].apply(lambda x: (x - test_data_norm['Close-Open_norm'].min()) / (test_data_norm['Close-Open_norm'].max() - test_data_norm['Close-Open_norm'].min()))

    # Remove the first and the last row, becuase of NAN values
    test_data_raw = test_data_raw.iloc[1:-1]
    test_data_norm = test_data_norm.iloc[1:-1]

    test_data_raw.to_csv(EXPORT_DATA_FOLDER+symbol+'test_raw_data.csv', index=True)
    test_data_norm.to_csv(EXPORT_DATA_FOLDER+symbol+'test_norm_data.csv', index=True)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


## TODO 

In [ ]:
'''
(2023-06-05)
cuda support check
//read textual data into correct shape
hyperparam tuning: number of neurons: tune to right number of neurons in FC in model
//max_text_per_iter -> code in dataloader to maintain the size 

(2023-06-07)
cuda check
roberta encoder fix
multi label - how to create target label?

'''

## Prep textual data

### Crawl textual news data from internet

In [ ]:
# Import libraries
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL', 'TSLA', 'AMZN']



# Get Data
finviz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finviz_url + ticker
    req = Request(url=url,
                  headers={'user-agent': 'Mozilla/5.0',
                                   'referer': 'https://...'}) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass


# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])
        


In [ ]:
parsed_news[0]

### Read downloaded data from saved files

In [6]:
text_data_df = pd.read_csv('./data/XOM_20200401_20230401_medium.csv', sep= ',', header= 0)
text_data_df = text_data_df[['Date', 'News']]


text_data_df = text_data_df.groupby('Date')['News'].apply('$$$###'.join)

text_data_df.index = pd.to_datetime(text_data_df.index, dayfirst=True)
# text_data_df


In [7]:
all_train_df = train_data_norm.join(text_data_df, how = 'inner')
all_test_df = test_data_norm.join(text_data_df, how = 'inner')

print(all_train_df.index.min())
print(all_train_df.index.max())
print(all_test_df.index.min())
print(all_test_df.index.max())

2020-04-01 00:00:00
2022-09-28 00:00:00
2022-10-04 00:00:00
2022-12-30 00:00:00


### Merge textual and numerical data


In [8]:
all_train = all_train_df.values

window_size = no_of_days_to_lookback

X_numerical_train = []
y_train = []
X_text_train = []
X_text_train_curr = []





for i in range(window_size, len(all_train) - no_of_days_to_lookforward + 1):
    X_numerical_train.append(all_train[i-window_size: i, :-2])
    
    # split and append sequence of text
    curr_seq = all_train[i-window_size: i, -1]
    for j in range(window_size):
        split_curr_seq = curr_seq[window_size - 1 -j].split('$$$###')
        X_text_train_curr = X_text_train_curr + split_curr_seq
    
    if len(X_text_train_curr) > max_text_per_iter:
        X_text_train_curr = X_text_train_curr[:max_text_per_iter]
    
    X_text_train.append(X_text_train_curr)
        
    # target labels
    y_train.append(all_train[i:i+no_of_days_to_lookforward, -2])

X_numerical_train, y_train = np.array(X_numerical_train).astype(np.float16), np.array(y_train).astype(np.int32)
print(type(X_numerical_train))
print(type(y_train))

X_numerical_train = torch.from_numpy(X_numerical_train).type(torch.Tensor)
y_train = torch.from_numpy(y_train).long()

print(len(X_numerical_train))
print(len(X_text_train))
print(len(y_train))
print(X_numerical_train.shape)

print(len(X_text_train))
print(len(X_text_train[2]))
# print(X_text_train[2])

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
624
624
624
torch.Size([624, 5, 8])
624
20


In [9]:
all_test = all_test_df.values


X_numerical_test = []
y_test = []
X_text_test = []
X_text_test_curr = []

for i in range(window_size, len(all_test) - no_of_days_to_lookforward + 1):
    X_numerical_test.append(all_test[i-window_size: i, :-2])
    
    # split and append sequence of text (in reverse order to add the latest news first)
    curr_seq = all_test[i-window_size: i, -1]
    for j in range(window_size):
        split_curr_seq = curr_seq[window_size - 1 -j].split('$$$###')
        X_text_test_curr = X_text_test_curr + split_curr_seq
    
    if len(X_text_test_curr) > max_text_per_iter:
        X_text_test_curr = X_text_test_curr[:max_text_per_iter]
    
    X_text_test.append(X_text_test_curr)
        
    # target labels
    y_test.append(all_test[i:i+no_of_days_to_lookforward, -2])

X_numerical_test, y_test = np.array(X_numerical_test).astype(np.float16), np.array(y_test).astype(np.int32)
print(type(X_numerical_test))
print(type(y_test))

X_numerical_test = torch.from_numpy(X_numerical_test).type(torch.Tensor)
y_test = torch.from_numpy(y_test).long()

print(len(X_numerical_test))
print(len(X_text_test))
print(len(y_test))
print(X_numerical_test.shape)

print(len(X_text_test))
print(len(X_text_test[2]))
# print(X_text_test[2])

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
57
57
57
torch.Size([57, 5, 8])
57
20


### Check sample train data

In [10]:
X_numerical_train[0]

tensor([[0.4246, 0.3257, 0.4453, 0.4441, 0.1932, 0.4934, 0.8872, 0.5684],
        [0.7788, 0.7954, 0.7358, 0.7979, 0.5088, 0.8218, 0.8267, 0.7310],
        [0.8540, 0.4360, 0.6265, 0.3726, 0.1608, 0.1741, 0.4355, 0.0930],
        [0.4275, 0.2666, 0.6353, 0.6196, 0.1874, 0.2355, 0.8696, 0.5459],
        [0.8765, 0.6895, 0.7617, 0.5669, 0.3101, 0.0968, 0.6763, 0.1940]])

In [11]:
len(X_text_train[0])
X_text_train[0]

['Exxon Cuts Capital Spending by 30% in Response to Coronavirus; The largest portion of the cuts will be in the Permian Basin, a key U.S. oil... Exxon Mobil Corp. said Tuesday that it would cut its 2020 capital spending by 30% as global demand for oil is sapped by the coronavirus. The largest portion of the $10 billion in cuts will be in the Permian Basin, the largest U.S. oil field ...',
 'Wall Street’s Rally Fades This briefing is no longer updating. Read the latest developments in the coronavirus outbreak here. The Treasury is trying to expand the small-business loans program.',
 'Coronavirus update: 1.39 million cases globally, 79,091 dead; Wuhan, China, reports zero deaths for first time since January; ExxonMobil... ExxonMobil cuts 2020 capex budget by 30% to combat oversupply and weak demand, while USA parent Gannett adopts poison pill The Chinese city of Wuhan reported zero deaths Tuesday from the coronavirus that causes COVID-19 for the first time ...',
 'ExxonMobil Reduces 202

In [15]:
y_train

tensor([[1],
        [0],
        [0],
        [1],
        [1],
        [2],
        [1],
        [0],
        [2],
        [2],
        [0],
        [0],
        [2],
        [2],
        [1],
        [1],
        [2],
        [0],
        [1],
        [0],
        [2],
        [0],
        [1],
        [1],
        [0],
        [0],
        [2],
        [1],
        [2],
        [1],
        [0],
        [2],
        [0],
        [1],
        [0],
        [2],
        [2],
        [2],
        [0],
        [2],
        [2],
        [1],
        [1],
        [1],
        [2],
        [0],
        [2],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [2],
        [1],
        [2],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [2],
        [0],
        [2],
        [0],
        [0],
        [1],
        [1],
        [2],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],

## Data loader

In [39]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

tokenizer = RobertaTokenizer.from_pretrained('roberta-large', truncation=True, do_lower_case=True)

class SiameseDataloader(Dataset):
    
    def __init__(self, X_numerical_train, y_train, X_text_train, MAX_LEN, tokenizer):
        self.X_numerical_train = X_numerical_train
        self.X_text_train = X_text_train
        self.MAX_LEN = MAX_LEN
        self.tokenizer = tokenizer
        
    def __getitem__(self, index):

        
        input_ids = []
        attention_masks = []
        token_type_ids = []
        input_seq = []

        for sent in X_text_train[index]:
            encoded_sent = self.tokenizer.encode_plus(
                text=sent,
                add_special_tokens=True,        # Add `[CLS]` and `[SEP]` special tokens
                max_length=self.MAX_LEN,             # Choose max length to truncate/pad
                pad_to_max_length=True,         # Pad sentence to max length 
                #return_attention_mask=True      # Return attention mask
                return_token_type_ids=True
                )
            input_ids.append(encoded_sent.get('input_ids'))
            attention_masks.append(encoded_sent.get('attention_mask'))
            token_type_ids.append(encoded_sent.get('token_type_ids'))

        # Convert lists to tensors
        input_ids = torch.tensor(input_ids)
        attention_masks = torch.tensor(attention_masks)
        token_type_ids = torch.tensor(token_type_ids)


        return {
            'x_numerical': X_numerical_train[index],
            'ids': torch.tensor(input_ids, dtype=torch.long),
            'mask': torch.tensor(attention_masks, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(y_train[index], dtype=torch.long)
        }
    
    
    
    

    def __len__(self):
        return len(self.X_numerical_train)

In [43]:
train_set = SiameseDataloader(X_numerical_train, y_train, X_text_train, MAX_LEN, tokenizer)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=False)


test_set = SiameseDataloader(X_numerical_test, y_test, X_text_test, MAX_LEN, tokenizer)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)


### Check the dataloader

In [44]:
for idx, data in tqdm(enumerate(train_loader, 0)):
    if idx > 1:
        break
        
    
    x_numerical = data['x_numerical'].to(device, dtype = torch.float)
    ids = data['ids'].to(device, dtype = torch.long)
    masks = data['mask'].to(device, dtype = torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
    targets = data['targets'].to(device, dtype = torch.long)
    



0it [00:00, ?it/s]/var/folders/tw/t5br5nkd2k3f065pr50zxvn80000gn/T/ipykernel_51835/3665585654.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'ids': torch.tensor(input_ids, dtype=torch.long),
/var/folders/tw/t5br5nkd2k3f065pr50zxvn80000gn/T/ipykernel_51835/3665585654.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'mask': torch.tensor(attention_masks, dtype=torch.long),
/var/folders/tw/t5br5nkd2k3f065pr50zxvn80000gn/T/ipykernel_51835/3665585654.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'token_type_ids': torch.tensor(t

In [45]:
print(x_numerical.shape)
x_numerical


torch.Size([8, 5, 8])


tensor([[[0.4482, 0.3164, 0.5532, 0.4597, 0.2559, 0.1724, 0.7974, 0.3616],
         [0.3933, 0.1324, 0.3101, 0.3062, 0.2703, 0.0145, 0.6846, 0.3652],
         [0.5269, 0.3479, 0.5117, 0.3589, 0.2173, 0.0000, 0.6943, 0.2324],
         [0.5186, 0.7314, 0.6587, 0.9082, 0.3604, 0.8525, 0.9980, 1.0000],
         [0.6636, 0.3513, 0.6284, 0.3013, 0.2277, 0.4827, 0.9590, 0.5181]],

        [[0.3933, 0.1324, 0.3101, 0.3062, 0.2703, 0.0145, 0.6846, 0.3652],
         [0.5269, 0.3479, 0.5117, 0.3589, 0.2173, 0.0000, 0.6943, 0.2324],
         [0.5186, 0.7314, 0.6587, 0.9082, 0.3604, 0.8525, 0.9980, 1.0000],
         [0.6636, 0.3513, 0.6284, 0.3013, 0.2277, 0.4827, 0.9590, 0.5181],
         [0.5435, 0.3472, 0.4666, 0.4692, 0.2900, 0.4233, 0.8477, 0.5400]],

        [[0.5269, 0.3479, 0.5117, 0.3589, 0.2173, 0.0000, 0.6943, 0.2324],
         [0.5186, 0.7314, 0.6587, 0.9082, 0.3604, 0.8525, 0.9980, 1.0000],
         [0.6636, 0.3513, 0.6284, 0.3013, 0.2277, 0.4827, 0.9590, 0.5181],
         [0.5435, 0.3

In [46]:
print(ids.shape)
ids


torch.Size([8, 20, 10])


tensor([[[    0,  9089, 43903,  ...,    30,   389,     2],
         [    0, 28216,   852,  ...,   274,  4216,     2],
         [    0, 15228,   261,  ...,   112,     4,     2],
         ...,
         [    0, 14773,  7458,  ...,  1782,   814,     2],
         [    0, 34543,   195,  ...,  5090,   389,     2],
         [    0,   698,    35,  ...,   189,   236,     2]],

        [[    0,  9089, 43903,  ...,    30,   389,     2],
         [    0, 28216,   852,  ...,   274,  4216,     2],
         [    0, 15228,   261,  ...,   112,     4,     2],
         ...,
         [    0, 14773,  7458,  ...,  1782,   814,     2],
         [    0, 34543,   195,  ...,  5090,   389,     2],
         [    0,   698,    35,  ...,   189,   236,     2]],

        [[    0,  9089, 43903,  ...,    30,   389,     2],
         [    0, 28216,   852,  ...,   274,  4216,     2],
         [    0, 15228,   261,  ...,   112,     4,     2],
         ...,
         [    0, 14773,  7458,  ...,  1782,   814,     2],
         [

In [47]:
print(targets.shape)
targets

torch.Size([8, 1])


tensor([[2],
        [2],
        [0],
        [0],
        [2],
        [2],
        [1],
        [1]])

## Build model


In [48]:



class SiameseModel(nn.Module):
    def __init__(self, input_dim1, input_dim2, 
                 hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4,
                 num_layers1, num_layers2, output_dim1, output_dim2):
        super(SiameseModel, self).__init__()
        self.input_dim1 = input_dim1
        self.input_dim2 = input_dim2
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2
        self.hidden_dim3 = hidden_dim3
        self.hidden_dim4 = hidden_dim4
        self.num_layers1 = num_layers1
        self.num_layers2 = num_layers2
        self.output_dim1 = output_dim1
        self.output_dim2 = output_dim2
        
        

#         self.roberta = RobertaModel.from_pretrained("roberta-large").to(device)
        
        
#         self.lstm1 = nn.LSTM(input_dim1, hidden_dim1, num_layers1, batch_first=True)
#         self.lstm2 = nn.LSTM(input_dim2, hidden_dim2, num_layers2, batch_first=True)


        self.fc1 = nn.Linear(hidden_dim1, output_dim1)
        self.fc2 = nn.Linear(hidden_dim2, output_dim2)
        self.fc3 = nn.Linear(input_dim1, hidden_dim3)
#         self.fc3 = nn.Linear(output_dim1+output_dim2, hidden_dim3)
        self.fc4 = nn.Linear(hidden_dim3, hidden_dim4)
        self.fc5 = nn.Linear(hidden_dim4, 3)
        
        
        
    def forward(self, x1, ids, masks, token_type_ids):
        #left tower with numerical features
#         h_10 = Variable(torch.zeros(self.num_layers1, x1.size(0), self.hidden_dim1)).to(device)
#         c_10 = Variable(torch.zeros(self.num_layers1, x1.size(0), self.hidden_dim1)).to(device)
#         ula1, (h_out1, _) = self.lstm1(x1, (h_10, c_10))
#         h_out1 = h_out1.view(-1, self.hidden_dim1)
#         out1 = self.fc1(h_out1)
        
        
        

        # right tower with roberta on textual features  
        #TODO
#         batch_size_here = ids.shape[0]
#         e2 = torch.zeros(batch_size_here, max_text_per_iter,1024).to(device)
        
#         for k in range(ids.shape[1]):
#             seq_ids = ids[:,k,:]
#             seq_masks = masks[:,k,:]
#             seq_token_type_ids = token_type_ids[:,k,:]


#             e2k = self.roberta(input_ids= seq_ids, attention_mask=seq_masks, token_type_ids=seq_token_type_ids)
#             # print(e2.shape)
#             # print(e2k[1].shape)
#             #first 0 is for last_hidden_state: https://huggingface.co/docs/transformers/main/en/model_doc/roberta#transformers.RobertaModel.forward.returns:~:text=transformers.modeling_outputs.-,BaseModelOutputWithPoolingAndCrossAttentions%20or%20tuple(torch.FloatTensor),-A%20transformers.modeling_outputs
#             # the shape of e2k[0] is (batch_size, sequence_length (<=MAX_LEN), hidden_size (=1024))
#             e2k1 = e2k[0][:, 0, :]  
#             e2[:,k,:] = e2k1
    
    
#         print('e2 shape: ', e2.shape)        
#         h_20 = Variable(torch.zeros(self.num_layers2, e2.size(0), self.hidden_dim2)).to(device)
#         c_20 = Variable(torch.zeros(self.num_layers2, e2.size(0), self.hidden_dim2)).to(device)
#         ula2, (h_out2, _) = self.lstm2(e2, (h_20, c_20))
#         h_out2 = h_out2.view(-1, self.hidden_dim2)
#         out2 = self.fc2(h_out2)
        
        
        
        
        # siamese merging layers
        x1 = torch.squeeze(x1[:, 0, :], 1) #x1[:, 0, :].squeeze()
        print('shape of x1 after squeeze:', x1.shape)
#         output = torch.cat((out1, out2),1)
#         output = F.relu(self.fc3(output))
        output = F.relu(self.fc3(x1))
        output = F.relu(self.fc4(output))
        output = self.fc5(output)
        return output
    
#TODO : correct these values
model = SiameseModel(input_dim1 = 8, input_dim2 = 1024, 
                 hidden_dim1 = 20, hidden_dim2 = 768, hidden_dim3 = 128, hidden_dim4 = 64,
                 num_layers1 = 1, num_layers2 = 1, output_dim1 = 10, output_dim2 = 256).to(device)


    
    


In [49]:
print(model)
print(len(list(model.parameters())))
# for i in range(len(list(model.parameters()))):
#     print(list(model.parameters())[i].size())


SiameseModel(
  (fc1): Linear(in_features=20, out_features=10, bias=True)
  (fc2): Linear(in_features=768, out_features=256, bias=True)
  (fc3): Linear(in_features=8, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=3, bias=True)
)
10


## Train model

In [58]:
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=0.008)
loss_arr = np.zeros(num_epochs)

In [59]:
wandb.init(project="stock_prediction")

avg test loss in this batch,█▇▆▆▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▁▂▃▃▃▃▃▄▄▄▄▄▅▅
avg test loss in this epoch,████████▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▄▄▄▄▃▃▃▂▂▂▂▁
avg train loss in this batch,▁▅▂▇▆▃▆▇▅▅▃▃▂▅▇▅▄█▃▃▄▄▇▁▆▂▆▁▁▆▇▃▅▃▃▅▆▆▄▄
avg train loss in this epoch,█████▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▁▁
batch_id,▅▁▁▁███▇▇▇▇▇▇▆▂▂▂▂▂▁▁▁▂▁▁▁█▄▄▄▃▃▃▃▃▂▂▂▂▂
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test accuracy in this epoch,▁▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▁▁███
avg test loss in this batch,0.65202
avg test loss in this epoch,1.0382
avg train loss in this batch,0.9309
avg train loss in this epoch,1.0041


In [ ]:
# roberta = RobertaModel.from_pretrained("roberta-large").to(device)
train_loss_record = []
for epoch in range(num_epochs):
    train_loss = []
    train_loss_sum = []
    train_correct = 0
    train_total = 0
    
    for idx, data in tqdm(enumerate(train_loader, 0)):
        x_numerical = data['x_numerical'].to(device, dtype = torch.float)
        ids = data['ids'].to(device, dtype = torch.long)
        masks = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        # debugging roberta encoder and second lstm
        '''
        debug starts here
        '''
        # if idx > 1:
        #     break
        # batch_size_here = data['ids'].shape[0]
        # print('batch_size_here:', batch_size_here)
        # e2 = torch.zeros(batch_size_here, max_text_per_iter, 1024)
        # print('ids shape:', ids.shape)
        
        # for k in range(ids.shape[1]):  #number of sentences in sequence = max_text_per_iter
        #     print('k:', k)
        #     seq_ids = ids[:,k,:].to(device)
        #     seq_masks = masks[:,k,:].to(device)
        #     seq_token_type_ids = token_type_ids[:,k,:].to(device)


        #     e2k = roberta(input_ids= seq_ids, attention_mask=seq_masks, token_type_ids=seq_token_type_ids)
        #     print(e2.shape)
        #     print(e2k[1].shape)
        #     #first 0 is for last_hidden_state: https://huggingface.co/docs/transformers/main/en/model_doc/roberta#transformers.RobertaModel.forward.returns:~:text=transformers.modeling_outputs.-,BaseModelOutputWithPoolingAndCrossAttentions%20or%20tuple(torch.FloatTensor),-A%20transformers.modeling_outputs
        #     # the shape of e2k[0] is (batch_size, sequence_length (<=MAX_LEN), hidden_size (=1024))
        #     e2k1 = e2k[0][:, 0, :]  
        #     e2[:,k,:] = e2k1
    
    
        

        # lstm2 = nn.LSTM(1024, 768, 1, batch_first=True)
        # fc2 = nn.Linear(768, 256)

        # h_20 = Variable(torch.zeros(1, e2.size(0), 768))
        # c_20 = Variable(torch.zeros(1, e2.size(0), 768))
        # ula2, (h_out2, _) = lstm2(e2, (h_20, c_20))
        # h_out2 = h_out2.view(-1, 768)
        # out2 = fc2(h_out2)
        

    #     print(ids.shape)
    #     print(masks.shape)
    #     print(token_type_ids.shape)
    
        # print(out2)


        
        '''
        debug ends here
        '''
        print('shape of x1 = x_numerical : ', x_numerical.shape)    
        y_pred = model(x_numerical, ids, masks, token_type_ids)
        _, pred_label = torch.max(y_pred.data, 1)

        print('y_pred:', y_pred)
        print('target:', targets)
        loss = criterion(y_pred, targets.reshape(-1))
        
         # Zero out gradient, else they will accumulate between epochs
        optimiser.zero_grad()

        # Backward pass
        loss.backward()

        # Update parameters
        optimiser.step()
        
        
        
        train_loss.append(loss.data.cpu())
        train_loss_sum.append(loss.data.cpu())
         

        wandb.log({'avg train loss in this batch': loss.item(), 'epoch': epoch, 'batch_id': idx})
        
        # Get accuracy
        train_total += targets.reshape(-1).size(0)
        train_correct += (pred_label == targets.reshape(-1)).sum()
        print('in this epoch, total, correct:', train_total, train_correct)
    train_accuracy = 100 * train_correct / train_total

    # Record at every epoch
    print('Train Loss at epoch {}: {}\n'.format(epoch, np.mean(train_loss_sum)))
    train_loss_record.append(np.mean(train_loss_sum))
    wandb.log({'avg train loss in this epoch': np.mean(train_loss_sum), 'epoch': epoch})
    wandb.log({'train accuracy in this epoch': train_accuracy, 'epoch': epoch})

    
    
    
    
    
    # evaluate on test set every epoch
    print('starting testing..')
    tloss = []
    test_loss_sum = []
    
    correct = 0
    total = 0
    
    for idx, data in tqdm(enumerate(test_loader, 0)):
        test_x_numerical = data['x_numerical'].to(device, dtype = torch.float)
        test_ids = data['ids'].to(device, dtype = torch.long)
        test_masks = data['mask'].to(device, dtype = torch.long)
        test_token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        test_targets = data['targets'].to(device, dtype = torch.long)
        
        print('shape of x1 = test_x_numerical : ', test_x_numerical.shape)    
        y_pred = model(test_x_numerical, test_ids, test_masks, test_token_type_ids)
        _, pred_label = torch.max(y_pred.data, 1)

#         print('y_pred:', y_pred)
        test_loss = criterion(y_pred, test_targets.reshape(-1))
    
        tloss.append(test_loss.data.cpu())
        test_loss_sum.append(test_loss.data.cpu()) 

        wandb.log({'avg test loss in this batch': test_loss.item(), 'epoch': epoch, 'batch_id': idx})
        
        # Get accuracy
        total += test_targets.reshape(-1).size(0)
        correct += (pred_label == test_targets.reshape(-1)).sum()
        print('in this epoch, total, correct:', total, correct)
    test_accuracy = 100 * correct / total

    # Record at every epoch
    print('test Loss at epoch {}: {}\n'.format(epoch, np.mean(test_loss_sum)))
    wandb.log({'avg test loss in this epoch': np.mean(test_loss_sum), 'epoch': epoch})
    wandb.log({'test accuracy in this epoch': test_accuracy, 'epoch': epoch})
    
    
    


            
    
        
        
        

0it [00:00, ?it/s]/var/folders/tw/t5br5nkd2k3f065pr50zxvn80000gn/T/ipykernel_51835/3665585654.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'ids': torch.tensor(input_ids, dtype=torch.long),
/var/folders/tw/t5br5nkd2k3f065pr50zxvn80000gn/T/ipykernel_51835/3665585654.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'mask': torch.tensor(attention_masks, dtype=torch.long),
/var/folders/tw/t5br5nkd2k3f065pr50zxvn80000gn/T/ipykernel_51835/3665585654.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'token_type_ids': torch.tensor(t

shape of x1 = x_numerical :  torch.Size([8, 5, 8])
shape of x1 after squeeze: torch.Size([8, 8])
y_pred: tensor([[ 0.4623, -0.0962, -0.3050],
        [ 1.2901, -0.7662, -0.9764],
        [ 0.1914,  0.0966, -0.2779],
        [ 0.4919, -0.1911, -0.2673],
        [-0.2352,  0.2587, -0.1040],
        [ 0.4931, -0.1054, -0.3419],
        [-0.4712,  0.2628, -0.0881],
        [ 0.5020, -0.2233, -0.2545]], grad_fn=<AddmmBackward0>)
target: tensor([[1],
        [0],
        [0],
        [1],
        [1],
        [2],
        [1],
        [0]])
in this epoch, total, correct: 8 tensor(5)
shape of x1 = x_numerical :  torch.Size([8, 5, 8])
shape of x1 after squeeze: torch.Size([8, 8])
y_pred: tensor([[ 0.5037, -0.2399, -0.2537],
        [ 0.2885, -0.2534, -0.1230],
        [ 0.3188, -0.1833, -0.1812],
        [ 1.0422, -0.4973, -0.8381],
        [ 0.4730, -0.1455, -0.2907],
        [ 0.4526, -0.0827, -0.3152],
        [ 0.3126,  0.0156, -0.3004],
        [ 0.6301, -0.3120, -0.3845]], grad_fn=<Addmm

4it [00:00,  9.04it/s]

shape of x1 = x_numerical :  torch.Size([8, 5, 8])
shape of x1 after squeeze: torch.Size([8, 8])
y_pred: tensor([[ 0.3972, -0.0413, -0.3100],
        [ 0.4751, -0.1689, -0.2776],
        [ 0.4743, -0.1665, -0.2786],
        [ 0.4713, -0.1571, -0.2823],
        [-0.2230, -0.0012,  0.0149],
        [ 0.5093, -0.2751, -0.2359],
        [ 0.4459, -0.0737, -0.3160],
        [ 0.0829,  0.1750, -0.2765]], grad_fn=<AddmmBackward0>)
target: tensor([[2],
        [0],
        [1],
        [0],
        [2],
        [0],
        [1],
        [1]])
in this epoch, total, correct: 24 tensor(12)
shape of x1 = x_numerical :  torch.Size([8, 5, 8])
shape of x1 after squeeze: torch.Size([8, 8])
y_pred: tensor([[ 5.0742e-01, -2.8536e-01, -2.2846e-01],
        [ 4.3724e-01, -1.6885e-01, -2.5859e-01],
        [ 4.6363e-01, -1.4916e-01, -2.8199e-01],
        [ 3.6615e-01, -2.4870e-01, -1.7799e-01],
        [ 2.9926e-01, -2.4316e-01, -1.4227e-01],
        [ 7.8222e-02, -2.7368e-01,  4.4799e-04],
        [ 4.374

6it [00:00,  9.11it/s]

...

29it [31:33, 11.49s/it]

shape of x1 = x_numerical :  torch.Size([8, 5, 8])
shape of x1 after squeeze: torch.Size([8, 8])
y_pred: tensor([[ 0.6133, -0.2477, -0.4800],
        [ 0.4587, -0.2455, -0.2130],
        [ 0.4104, -0.0940, -0.3311],
        [ 0.3787, -0.1349, -0.2709],
        [ 0.4450, -0.2267, -0.2229],
        [ 0.4786, -0.2627, -0.2089],
        [ 0.4573, -0.2436, -0.2140],
        [ 0.4855, -0.2150, -0.2545]], grad_fn=<AddmmBackward0>)
target: tensor([[0],
        [0],
        [2],
        [2],
        [0],
        [1],
        [2],
        [0]])
in this epoch, total, correct: 232 tensor(108)


31it [31:33,  5.73s/it]

shape of x1 = x_numerical :  torch.Size([8, 5, 8])
shape of x1 after squeeze: torch.Size([8, 8])
y_pred: tensor([[ 0.5410, -0.2297, -0.3539],
        [ 0.4823, -0.2144, -0.2506],
        [ 0.4295, -0.2108, -0.2259],
        [ 0.4586, -0.2509, -0.2051],
        [ 0.4466, -0.1573, -0.2865],
        [ 0.4743, -0.2679, -0.1992],
        [ 0.4743, -0.2679, -0.1992],
        [ 0.3768, -0.2317, -0.1744]], grad_fn=<AddmmBackward0>)
target: tensor([[0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0]])
in this epoch, total, correct: 240 tensor(113)
shape of x1 = x_numerical :  torch.Size([8, 5, 8])
shape of x1 after squeeze: torch.Size([8, 8])
y_pred: tensor([[ 0.3999, -0.1730, -0.2423],
        [ 0.4721, -0.2193, -0.2396],
        [ 0.2254, -0.0345, -0.2566],
        [ 0.5009, -0.3874, -0.3222],
        [ 0.4716, -0.2670, -0.1965],
        [ 0.4539, -0.1719, -0.2742],
        [ 0.4652, -0.1898, -0.2630],
        [ 0.4508, -0.1669, -0.2774]], grad_fn=<A

32it [31:33,  4.10s/it]

In [ ]:
print(test_targets.reshape(-1))

In [ ]:
_, pred_label = torch.max(y_pred.data, 1)
pred_label

In [ ]:
(pred_label == test_targets.reshape(-1)).sum()

In [ ]:
run_id = str(1)
!mkdir output/$run_id
output_model_file = 'roberta_stock_pred.bin'
output_vocab_file = './output' + str(run_id)

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)


In [ ]:
plt.plot(loss_arr, label="Training loss")
plt.legend()
plt.show()

In [ ]:

'''